In [411]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [412]:
%matplotlib notebook

In [413]:
%sql sqlite:///schema/locate_tuesday.db

u'Connected: None@schema/locate_tuesday.db'

In [414]:
import locate
import farwave
import numpy as np
import scipy as sp

%sql select * from experiment;

Done.


id,datetime,x,y,x_hat,y_hat,comment
1,1489511414,0.0,2.0,-0.0371689346478,1.45508694295,
2,1489511437,0.0,2.0,-0.0648942819059,1.47057537897,
3,1489511460,0.0,2.0,0.000978633106023,1.47566857787,
4,1489511564,-3.0,0.0,0.751690140398,-0.355342680301,
5,1489511588,-3.0,0.0,1.06067103748,-0.45014595152,
6,1489511610,-3.0,0.0,0.998095565523,-0.380003413791,
7,1489512539,-1.0,2.0,17.2769732824,-51.6906023636,
8,1489512564,-1.0,2.0,-5.07182571152,5.89501343973,
9,1489512587,-1.0,2.0,-1.20469889381,3.32741564489,
10,1489512640,-1.0,2.0,2.91073220162,-3.3491645856,


In [415]:
l = 0.3 # Length from mic to centre of array
MIC_IND_LR = {
    0: (1, 2),
    1: (2, 0),
    2: (0, 1)
}

ANGLE_OFFSET = {
    0: 0,
    1: 2.0944, # 120 deg in rad
    2: 4.18879, # 240 deg in rad
}

def calc_delays(sigs, offsets):
    """ Calculate matrix of delays given N sigs
    """
    results = []
    for j in range(len(sigs)):
        for k in range(len(sigs)):
            if j != k:
                results.append((
                        (j, k), locate.xcorr_peaks(sigs[j], sigs[k], offsets[j], offsets[k], l)
                    ))
            

    # 3x3 array delays[i][j] is the delay of signal j relative to signal i
    delays = np.zeros((len(sigs), len(sigs)), dtype=np.float)
    for res in results:
        key, val = res[0], res[1]
        delays[key[0]][key[1]] = val[1] # xcorr (val, delay) tuple
        delays[key[1]][key[0]] = -delays[key[0]][key[1]]
        
    return delays

def calc_angles(n, delays, l):
    """ Given delays matrix, calculate angle using far_wave
    """
    # Estimate "location" of sound source, create array record
    for j in range(n):
        if delays[j][(j+1)%3] >= 0 and delays[j][(j+2)%3] >= 0:
            lr = MIC_IND_LR[j]
            near_pair = (min(j, lr[0]), max(j, lr[0])) if delays[j][lr[0]] < delays[j][lr[1]] else (min(j, lr[1]), max(j, lr[1]))
            print 'Near Pair: %r' %  str(near_pair)
            f_theta = np.deg2rad(farwave.calc_angle(delays, l, near_pair=near_pair))
            print 'Using microphone %d as closest mic - (%d left, %d right)' % (j, lr[0], lr[1])
            break
    
    return f_theta, np.rad2deg(f_theta)


In [416]:
cur_experiment_id = 19

result = %sql select data from mic where experiment_id = :cur_experiment_id and array_id = 0
    
mic_0a = np.array(result[0][0])
mic_1a = np.array(result[1][0])
mic_2a = np.array(result[2][0])

sigs_a = [mic_0a, mic_1a, mic_2a]

result = %sql select data from mic where experiment_id = :cur_experiment_id and array_id = 1
    
mic_0b = np.array(result[0][0])
mic_1b = np.array(result[1][0])
mic_2b = np.array(result[2][0])

sigs_b = [mic_0b, mic_1b, mic_2b]

Done.
Done.


In [417]:
_, a_crop, a_off, _, _ = locate.crop_sigs_npeaks(sigs_a)
_, b_crop, b_off, _, _ = locate.crop_sigs_npeaks(sigs_b)

In [418]:
del_a = calc_delays(a_crop, a_off)
theta_a, theta_a_deg = calc_angles(len(sigs_a), del_a, l)

del_b = calc_delays(b_crop, b_off)
theta_b, theta_b_deg = calc_angles(len(sigs_b), del_b, l)

Near Pair: '(0, 2)'
Angles: [257.26079924469252, 286.48504051135797, 328.73565746599689]
Errors: [1.0471598549619132, 1.0284796966276135, 1.1698889550207043]

DEL_ERR: 0.018680
Error: 0.018680, 1.028480, 'True'
Setting final_ang to 286.485041
DEL_ERR: 0.141409
Error: 0.141409, 1.028480, 'True'
Setting final_ang to 286.485041
Angle (CCW): 286.485041
Using microphone 2 as closest mic - (0 left, 1 right)
Near Pair: '(0, 2)'
Angles: [282.42689764714243, 305.97772619815305, 325.21440161844146]
Errors: [1.3547589246257403, 1.0054672727565632, 1.2175928007223327]

DEL_ERR: 0.349292
Error: 0.349292, 1.005467, 'True'
DEL_ERR: 0.137166
Error: 0.137166, 1.005467, 'False'
DEL_ERR: 0.212126
Error: 0.212126, 1.005467, 'True'
Angle (CCW): 305.573315
Using microphone 0 as closest mic - (1 left, 2 right)


In [419]:
results = %sql select array_id, x, y from array where experiment_id = :cur_experiment_id
    
print results

Done.
+----------+---+---+
| array_id | x | y |
+----------+---+---+
+----------+---+---+


In [420]:
R = np.array([[0, -1], [1, 0]])

p_a = np.array([0, 0]) #np.array([results[0]['x'], results[0]['y']])
p_b = np.array([2, 0]) #np.array([results[1]['x'], results[1]['y']])

v_a = np.dot(R, np.array([np.cos(theta_a), np.sin(theta_a)]))
v_b = np.dot(R, np.array([np.cos(theta_b), np.sin(theta_b)]))

calc_pos = locate.calc_poi(
    p_a,
    p_b,
    v_a,
    v_b
)

print theta_a, theta_b
print p_a, v_a
print p_b, v_b

5.00010721463 5.33326045799
[0 0] [ 0.95889386  0.28376499]
[2 0] [ 0.81337179  0.58174422]


In [421]:
results = %sql select x, y from experiment where id = :cur_experiment_id

Done.


In [422]:
act_pos = np.array([results[0]['x'], results[0]['y']])

In [423]:
print 'Computed: %s' % str(calc_pos)
print 'Actual: %s' % str(act_pos)
print 'Error: %f' % (np.sqrt(np.sum((calc_pos - act_pos)**2)))


print del_a
print del_b

Computed: [ 3.41155437  1.00957963]
Actual: [ 4.  2.]
Error: 1.152042
[[    0.  4308. -3393.]
 [-4308.     0. -7535.]
 [ 3393.  7535.     0.]]
[[    0.  9795.  1512.]
 [-9795.     0. -8283.]
 [-1512.  8283.     0.]]
